In [5]:
import pandas as pd

df = pd.read_csv("drive_download/summarization_data_newsroom.csv")
df = df.drop(columns=['density_bin'])
df

,text,summary
0,By MATT SCHWARTZ in Houston and WENDELL JAMIES...,"Bleeding from a massive chest wound, Tejano st..."
1,"By HOLLY RAMER, Associated Press\n\nCONCORD, N...","By HOLLY RAMER, Associated Press CONCORD, N.H...."
2,Men in battle-fatigues have raided the Moscow ...,Men in battle-fatigues have&nbsp;raided the Mo...
3,"Like all gadgets, cellphones can break. In fac...",Simple home remedies for repairing your mobile...
4,"I visited Saudi Arabia in September 2008, arri...","Hurricane Ike, Ramadan and the billionaire pri..."
...,...,...
108857,"Ahumdinger TV season wrapped Wednesday night, ...","Ahumdinger TV season wrapped Wednesday night, ..."
108858,A Senate panel reached bipartisan agreement on...,A Senate panel reached bipartisan agreement on...
108859,In a dramatic break with the ideological warfa...,"Fragile bipartisan compromise, at best, has pr..."
108860,"In 1967, LaDonna Davis's boyfriend went on a t...",Get style news headlines from The Washington P...


In [6]:
from summarizer import Summarizer
model = Summarizer('distilbert-base-uncased')

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

In [7]:
def zsm_df(txt):
    zsm_txt = model(txt, ratio=0.6)
    return zsm_txt

In [20]:
df = df[:5000]

In [21]:
df

,text,summary
0,By MATT SCHWARTZ in Houston and WENDELL JAMIES...,"Bleeding from a massive chest wound, Tejano st..."
1,"By HOLLY RAMER, Associated Press\n\nCONCORD, N...","By HOLLY RAMER, Associated Press CONCORD, N.H...."
2,Men in battle-fatigues have raided the Moscow ...,Men in battle-fatigues have&nbsp;raided the Mo...
3,"Like all gadgets, cellphones can break. In fac...",Simple home remedies for repairing your mobile...
4,"I visited Saudi Arabia in September 2008, arri...","Hurricane Ike, Ramadan and the billionaire pri..."
...,...,...
4995,A masked gunman has been shot dead by police a...,A man has reportedly opened fire inside a cine...
4996,We are continually improving the quality of ou...,"Davidson, Genevieve Sullivan (Mrs)"
4997,can finally breathe easy -- he is officially d...,Antoine Walker can finally breathe easy -- he ...
4998,Photo: Kimberly Veklerov / /\n\nReps. Jared Hu...,"California commercial crab fishermen, many str..."


In [22]:
df["zsm"] = df.apply(lambda row : zsm_df(row["text"]), axis = 1)

/home/rechcons/PP3-NLP/.PPP-NLP-env/lib/python3.8/site-packages/summarizer/cluster_features.py:149: ConvergenceWarning: Number of distinct clusters (132) found smaller than n_clusters (182). Possibly due to duplicate points in X.
  model = self._get_model(k).fit(self.features)
/home/rechcons/PP3-NLP/.PPP-NLP-env/lib/python3.8/site-packages/summarizer/cluster_features.py:149: ConvergenceWarning: Number of distinct clusters (42) found smaller than n_clusters (49). Possibly due to duplicate points in X.
  model = self._get_model(k).fit(self.features)
/home/rechcons/PP3-NLP/.PPP-NLP-env/lib/python3.8/site-packages/summarizer/cluster_features.py:149: ConvergenceWarning: Number of distinct clusters (57) found smaller than n_clusters (70). Possibly due to duplicate points in X.
  model = self._get_model(k).fit(self.features)
/home/rechcons/PP3-NLP/.PPP-NLP-env/lib/python3.8/site-packages/summarizer/cluster_features.py:149: ConvergenceWarning: Number of distinct clusters (65) found smaller tha

In [65]:
df = df.drop(index=df[df["zsm"]==""].index).reset_index(drop=True)

In [70]:
from rouge import Rouge 

def rogue_scorer(ypred,ytrue):
    rouge = Rouge()
    scores = rouge.get_scores(ypred, ytrue)
    return scores[0]["rouge-1"]

In [71]:
x = df.apply(lambda row : rogue_scorer(row["zsm"],row["summary"]), axis = 1)

In [72]:
x

0       {'r': 0.9354838709677419, 'p': 0.3276836158192...
1       {'r': 1.0, 'p': 0.31451612903225806, 'f': 0.47...
2       {'r': 0.8181818181818182, 'p': 0.0608108108108...
3       {'r': 0.15384615384615385, 'p': 0.052631578947...
4       {'r': 0.5714285714285714, 'p': 0.0195121951219...
                              ...                        
4989    {'r': 0.5333333333333333, 'p': 0.0672268907563...
4990                       {'r': 0.0, 'p': 0.0, 'f': 0.0}
4991    {'r': 0.5769230769230769, 'p': 0.3409090909090...
4992    {'r': 0.9004149377593361, 'p': 0.434, 'f': 0.5...
4993    {'r': 0.8571428571428571, 'p': 0.0192926045016...
Length: 4994, dtype: object

In [74]:
df = df.join(pd.json_normalize(x))

In [77]:
df

,text,summary,zsm,r,p,f
0,By MATT SCHWARTZ in Houston and WENDELL JAMIES...,"Bleeding from a massive chest wound, Tejano st...",By MATT SCHWARTZ in Houston and WENDELL JAMIES...,0.935484,0.327684,0.485356
1,"By HOLLY RAMER, Associated Press\n\nCONCORD, N...","By HOLLY RAMER, Associated Press CONCORD, N.H....","By HOLLY RAMER, Associated Press\n\nCONCORD, N...",1.000000,0.314516,0.478528
2,Men in battle-fatigues have raided the Moscow ...,Men in battle-fatigues have&nbsp;raided the Mo...,Men in battle-fatigues have raided the Moscow ...,0.818182,0.060811,0.113208
3,"Like all gadgets, cellphones can break. In fac...",Simple home remedies for repairing your mobile...,"In fact, our habit of carrying our phones cons...",0.153846,0.052632,0.078431
4,"I visited Saudi Arabia in September 2008, arri...","Hurricane Ike, Ramadan and the billionaire pri...","I visited Saudi Arabia in September 2008, arri...",0.571429,0.019512,0.037736
...,...,...,...,...,...,...
4989,A masked gunman has been shot dead by police a...,A man has reportedly opened fire inside a cine...,A masked gunman has been shot dead by police a...,0.533333,0.067227,0.119403
4990,We are continually improving the quality of ou...,"Davidson, Genevieve Sullivan (Mrs)",We are continually improving the quality of ou...,0.000000,0.000000,0.000000
4991,can finally breathe easy -- he is officially d...,Antoine Walker can finally breathe easy -- he ...,can finally breathe easy -- he is officially d...,0.576923,0.340909,0.428571
4992,Photo: Kimberly Veklerov / /\n\nReps. Jared Hu...,"California commercial crab fishermen, many str...",Jared Huffman and Jackie Speier speak at a new...,0.900415,0.434000,0.585695


In [76]:
print(df.r.mean(),df.p.mean(),df.f.mean())

0.6138142640577363 0.12527889432951522 0.19087953586473355


---

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained('t5-base')
tokenizer = AutoTokenizer.from_pretrained('t5-base')

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

/home/rechcons/PP3-NLP/.PPP-NLP-env/lib/python3.8/site-packages/transformers/models/t5/tokenization_t5_fast.py:156: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [5]:
text = """New York (CNN Business)Netflix is synonymous with streaming, but its competitors have a distinct advantage that threatens the streaming leader's position at the top.
Disney has Disney+, but it also has theme parks, plush Baby Yoda dolls, blockbuster Marvel movies and ESPN. Comcast (CMCSA), Amazon (AMZN), ViacomCBS (VIACA), CNN's parent company WarnerMedia and Apple (AAPL) all have their own streaming services, too, but they also have other forms of revenue.
As for Netflix (NFLX), its revenue driver is based entirely on building its subscriber base. It's worked out well for the company — so far. But it's starting to look like the king of streaming will soon need something other than new subscribers to keep growing.
The streaming service reported Tuesday it now has 208 million subscribers globally, after adding 4 million subscribers in the first quarter of 2021. But that number missed expectations and the forecasts for its next quarter were also pretty weak.
That was a big whiff for Netflix — a company coming off a massive year of growth thanks in large part to the pandemic driving people indoors — and Wall Street's reaction has not been great.
The company's stock dropped as much as 8% on Wednesday, leading some to wonder what the future of the streamer looks like if competition continues to gain strength, people start heading outdoors and if, most importantly, its growth slows.
"If you hit a wall with [subscriptions] then you pretty much don't have a super growth strategy anymore in your most developed markets," Michael Nathanson, a media analyst and founding partner at MoffettNathanson, told CNN Business. "What can they do to take even more revenue out of the market, above and beyond streaming revenues?"
Or put another way, the company's lackluster user growth last quarter is a signal that it wouldn't hurt if Netflix — a company that's lived and died with its subscriber numbers — started thinking about other ways to make money.
An ad-supported Netflix? Not so fast
There are ways for Netflix to make money other than raising prices or adding subscribers. The most obvious: selling advertising.
Netflix could have 30-second commercials on their programming or get sponsors for their biggest series and films. TV has worked that way forever, why not Netflix?
That's probably not going to happen, given that CEO Reed Hastings has been vocal about the unlikelihood of an ad-supported Netflix service. His reasoning: It doesn't make business sense.
"It's a judgment call... It's a belief we can build a better business, a more valuable business [without advertising]," Hastings told Variety in September. "You know, advertising looks easy until you get in it. Then you realize you have to rip that revenue away from other places because the total ad market isn't growing, and in fact right now it's shrinking. It's hand-to-hand combat to get people to spend less on, you know, ABC and to spend more on Netflix."
Hastings added that "there's much more growth in the consumer market than there is in advertising, which is pretty flat."
He's also expressed doubts about Netflix getting into live sports or news, which could boost the service's allure to subscribers, so that's likely out, too, at least for now.
So if Netflix is looking for other forms of near-term revenue to help support its hefty content budget ($17 billion in 2021 alone) then what can it do? There is one place that could be a revenue driver for Netflix, but if you're borrowing your mother's account you won't like it.
Netflix could crack down on password sharing — a move that the company has been considering lately.
"Basically you're going to clean up some subscribers that are free riders," Nathanson said. "That's going to help them get to a higher level of penetration, definitely, but not in long-term."
Lackluster growth is still growth
Missing projections is never good, but it's hardly the end of the world for Netflix. The company remains the market leader and most competitors are still far from taking the company on. And while Netflix's first-quarter subscriber growth wasn't great, and its forecasts for the next quarter alarmed investors, it was just one quarter.
Netflix has had subscriber misses before and it's still the most dominant name in all of streaming, and even lackluster growth is still growth. It's not as if people are canceling Netflix in droves.
Asked about Netflix's "second act" during the company's post-earnings call on Tuesday, Hastings again placed the company's focus on pleasing subscribers.
"We do want to expand. We used to do that thing shipping DVDs, and luckily we didn't get stuck with that. We didn't define that as the main thing. We define entertainment as the main thing," Hastings said.
He added that he doesn't think Netflix will have a second act in the way Amazon has had with Amazon shopping and Amazon Web Services. Rather, Netflix will continue to improve and grow on what it already does best.
"I'll bet we end with one hopefully gigantic, hopefully defensible profit pool, and continue to improve the service for our members," he said. "I wouldn't look for any large secondary pool of profits. There will be a bunch of supporting pools, like consumer products, that can be both profitable and can support the title brands."""

In [7]:
print(tokenizer.model_max_length)

512


In [24]:
import math
mole = tokenizer.model_max_length
x = math.ceil(len(text.split())/mole)

In [96]:
compression = 0.1

ftextlen = math.ceil((1-compression)*len(text.split()))




In [97]:
textlist = []
for i in range(1,x+1):
    txt = " ".join(str(x) for x in text.split()[mole*(i-1):mole*i])
    tokens_input = tokenizer.encode("summarize: " + txt,
                                return_tensors='pt',
                                max_length=mole,
                                )
    
    wahrs = round(len(txt.split())/len(text.split()),2)
    print(f"Min Length:{math.ceil((wahrs*ftextlen)-(wahrs*ftextlen)*0.05)}, Max length:{math.ceil((wahrs*ftextlen)+(wahrs*ftextlen)*0.05)}")
    summary_ids = model.generate(tokens_input,  min_length=math.ceil((wahrs*ftextlen)-(wahrs*ftextlen)*0.05), 
                                                max_length=math.ceil((wahrs*ftextlen)+(wahrs*ftextlen)*0.05),num_beams=2)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    print(len(summary.split()))
    textlist.append(summary)
finaltxt = " ".join(str(x) for x in textlist)

Min Length:437, Max length:483


335
Min Length:317, Max length:350
196


In [93]:
len(finaltxt.split())

183

In [94]:
ftextlen

440

In [95]:
len(text.split())

879